In [7]:
import pandas as pd
df_csv = pd.read_csv("housing.csv") # Leer el CSV
print(df_csv.head())



   longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
0    -122.23     37.88                41.0        880.0           129.0   
1    -122.22     37.86                21.0       7099.0          1106.0   
2    -122.24     37.85                52.0       1467.0           190.0   
3    -122.25     37.85                52.0       1274.0           235.0   
4    -122.25     37.85                52.0       1627.0           280.0   

   population  households  median_income  median_house_value ocean_proximity  
0       322.0       126.0         8.3252            452600.0        NEAR BAY  
1      2401.0      1138.0         8.3014            358500.0        NEAR BAY  
2       496.0       177.0         7.2574            352100.0        NEAR BAY  
3       558.0       219.0         5.6431            341300.0        NEAR BAY  
4       565.0       259.0         3.8462            342200.0        NEAR BAY  


In [8]:
import pandas as pd

# 1️⃣ Cargar el archivo de collab ya subido
df = pd.read_csv("housing.csv")

# 2️⃣ Ver columnas originales
print("Columnas originales:")
print(df.columns.tolist())

# 3️⃣ Conservar solo las columnas deseadas
cols = ["longitude", "latitude", "median_house_value"]
df = df[cols]

# 4️⃣ Revisar valores nulos
print("\nValores nulos por columna:")
print(df.isnull().sum())

# 5️⃣ Comprobar si las coordenadas están dentro del rango típico de California
out_long = ((df["longitude"] < -125) | (df["longitude"] > -113)).sum()
out_lat = ((df["latitude"] < 32) | (df["latitude"] > 42)).sum()

#Los valores −125 y −113 utilizados para la longitud, así como 32 y 42 para la latitud, provienen del rango geográfico real del estado de California, meridano de GAREENwICH y línea del ecuador.

print(f"\nValores fuera de rango → longitude: {out_long}, latitude: {out_lat}")

# 6️⃣ Limpiar datos (eliminar nulos y fuera de rango)
df_clean = df.dropna()
df_clean = df_clean[
    (df_clean["longitude"].between(-125, -113)) &
    (df_clean["latitude"].between(32, 42))
]

print(f"\nRegistros válidos después de limpieza: {len(df_clean)} de {len(df)}")

# 7️⃣ Guardar el dataset limpio
df_clean.to_csv("housing_limpio.csv", index=False)
print("\n✅ Archivo limpio guardado como 'housing_limpio.csv'")


Columnas originales:
['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'median_house_value', 'ocean_proximity']

Valores nulos por columna:
longitude             0
latitude              0
median_house_value    0
dtype: int64

Valores fuera de rango → longitude: 0, latitude: 0

Registros válidos después de limpieza: 20640 de 20640

✅ Archivo limpio guardado como 'housing_limpio.csv'


In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error

# 1. Cargar dataset
df = pd.read_csv("housing_limpio.csv")

# 2. Separar features y target
X = df[["longitude", "latitude"]]   # puedes agregar más columnas si quieres
y = df["median_house_value"]

# 3. Dividir en train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1
)

# 4. Escalar datos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 5. Montecarlo: probar configuraciones aleatorias
n_iter = 20   # número de simulaciones
mejor_rmse = float("inf")
mejor_config = None

for i in range(n_iter):
    # 🔹 Menor cantidad de neuronas (más apropiado para 2-3 variables)
    hidden1 = np.random.randint(5, 30)    # capa 1 entre 5 y 30 neuronas
    hidden2 = np.random.randint(3, 15)    # capa 2 entre 3 y 15 neuronas
    activation = np.random.choice(["relu", "tanh", "logistic"])
    max_iter = np.random.randint(500, 2000)

    # Definir modelo con esos hiperparámetros
    regr = MLPRegressor(
        hidden_layer_sizes=(hidden1, hidden2),
        activation=activation,
        solver="adam",
        max_iter=max_iter,
        random_state=1,
        early_stopping=True  # detiene si deja de mejorar
    )

    # Entrenar
    regr.fit(X_train, y_train)

    # Evaluar
    y_pred = regr.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = regr.score(X_test, y_test)

    print(f"Iteración {i+1}: capas=({hidden1},{hidden2}), act={activation}, "
          f"max_iter={max_iter}, RMSE={rmse:.2f}, R²={r2:.3f}")

    # Guardar mejor resultado
    if rmse < mejor_rmse:
        mejor_rmse = rmse
        mejor_config = (hidden1, hidden2, activation, max_iter, r2)

print("\n🔹 Mejor configuración encontrada:")
print(f"Capas ocultas: {mejor_config[0]}, {mejor_config[1]}")
print(f"Activación: {mejor_config[2]}")
print(f"Max_iter: {mejor_config[3]}")
print(f"RMSE: {mejor_rmse:.2f}, R²: {mejor_config[4]:.3f}")




Iteración 1: capas=(9,12), act=tanh, max_iter=1992, RMSE=233358.53, R²=-3.152
Iteración 2: capas=(8,9), act=tanh, max_iter=1610, RMSE=233361.94, R²=-3.152
Iteración 3: capas=(9,4), act=logistic, max_iter=797, RMSE=233369.35, R²=-3.152
Iteración 4: capas=(19,8), act=relu, max_iter=772, RMSE=99591.57, R²=0.244
Iteración 5: capas=(9,12), act=logistic, max_iter=817, RMSE=233363.31, R²=-3.152
Iteración 6: capas=(19,11), act=tanh, max_iter=659, RMSE=233360.43, R²=-3.152
Iteración 7: capas=(20,14), act=relu, max_iter=1566, RMSE=98602.19, R²=0.259
Iteración 8: capas=(12,5), act=relu, max_iter=698, RMSE=99530.34, R²=0.245
Iteración 9: capas=(6,5), act=logistic, max_iter=1375, RMSE=233370.91, R²=-3.152
Iteración 10: capas=(13,4), act=logistic, max_iter=668, RMSE=233370.99, R²=-3.152
Iteración 11: capas=(23,8), act=tanh, max_iter=774, RMSE=233364.20, R²=-3.152
Iteración 12: capas=(10,13), act=tanh, max_iter=1755, RMSE=233358.98, R²=-3.152
Iteración 13: capas=(15,9), act=tanh, max_iter=1062, RMSE=

In [12]:
import joblib
joblib.dump(regr, 'mlp_model.pkl')
print("Modelo MLP exportado correctamente")

Modelo MLP exportado correctamente
